In [1]:
import os
import glob
import random
import librosa
import librosa.display 
import numpy as np
import pandas as pd

import tensorflow 
import IPython.display as ipd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

%matplotlib inline

random.seed=(123)

In [2]:
data = ImageDataGenerator()
data = data.flow_from_directory(directory="../data/test", target_size=(224,224),shuffle=False, batch_size=32) 

Found 2203 images belonging to 100 classes.


In [3]:
d_lab = data.class_indices

In [4]:
BATCH_SIZE = 32
tedata = ImageDataGenerator()

testdata = tedata.flow_from_directory(directory="../data/source/Source",target_size=(224,224) ,shuffle=False, batch_size=BATCH_SIZE)

Found 100 images belonging to 66 classes.


In [5]:
model_name = "../models/cnn_20epoch.h5"

In [6]:
from tensorflow.keras.models import load_model
model = load_model(model_name)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 128)      

In [7]:
output=model.predict(testdata,verbose=1)
predicted=np.argmax(output,axis=1)

d_lab = dict((v,k) for k,v in d_lab.items())

4/4 [==============================] - 18s 4s/step


In [8]:
predictions = [d_lab[k] for k in predicted]

In [9]:
filenames=testdata.filenames
sspker_label = [name[name.find("sp"):name.find("/sp")] for name in filenames]
print(sspker_label)

['sp0032', 'sp0032', 'sp0093', 'sp0093', 'sp0112', 'sp0240', 'sp0296', 'sp0296', 'sp0459', 'sp0459', 'sp0472', 'sp0472', 'sp0492', 'sp0597', 'sp0636', 'sp0637', 'sp1050', 'sp1050', 'sp1066', 'sp1066', 'sp1112', 'sp1182', 'sp1182', 'sp1246', 'sp1271', 'sp1271', 'sp1392', 'sp1392', 'sp1417', 'sp1472', 'sp1536', 'sp1607', 'sp1607', 'sp1607', 'sp1841', 'sp1926', 'sp2012', 'sp2012', 'sp2156', 'sp2269', 'sp2269', 'sp2289', 'sp2289', 'sp2289', 'sp2294', 'sp2412', 'sp2532', 'sp2532', 'sp2673', 'sp2803', 'sp3235', 'sp3446', 'sp3446', 'sp3521', 'sp3645', 'sp3835', 'sp3835', 'sp3923', 'sp4057', 'sp4057', 'sp4057', 'sp4427', 'sp4427', 'sp4590', 'sp4744', 'sp4839', 'sp4848', 'sp4848', 'sp4957', 'sp4957', 'sp4957', 'sp5319', 'sp5583', 'sp5583', 'sp5717', 'sp5717', 'sp5789', 'sp5968', 'sp6099', 'sp6415', 'sp6544', 'sp6574', 'sp6965', 'sp7000', 'sp7000', 'sp7000', 'sp7517', 'sp7517', 'sp7517', 'sp7688', 'sp7850', 'sp7868', 'sp7881', 'sp7976', 'sp7976', 'sp8425', 'sp8635', 'sp8635', 'sp8713', 'sp8713']

In [10]:
results=pd.DataFrame({"Filename":sspker_label,
                      "Predictions":predictions})

In [11]:
corr = results.loc[results['Filename'] == results['Predictions']]
(len(corr)/len(results)) * 100

93.0

In [12]:
results

,Filename,Predictions
0,sp0032,sp0032
1,sp0032,sp0032
2,sp0093,sp0093
3,sp0093,sp0093
4,sp0112,sp0112
...,...,...
95,sp8425,sp8425
96,sp8635,sp8635
97,sp8635,sp8635
98,sp8713,sp8713


In [13]:
f_lst = glob.glob("../data/source/Source/*/*.png")
sspker = [name[name.find("/sp")+8:name.find("-ch")] for name in f_lst]

In [15]:
out = [] 
d_lab = dict((v,k) for k,v in d_lab.items())
from tensorflow.keras.preprocessing import image
for i in f_lst:
    img = image.load_img(i,target_size=(224,224))
    img = np.asarray(img)
    img = np.expand_dims(img, axis=0)
    output = model.predict(img)
    out.append(d_lab[output.argmax()])

In [16]:
img_result = [] 
for i in range(len(out)):
    img_result.append([out[i], sspker[i]])

In [17]:
count = 0 
for i in range(len(img_result)):
    if img_result[i][0] == img_result[i][1]:
        count += 1

In [18]:
count/len(img_result)

0.93